## Cleaning the Data
-----

### Setup:
---

In [192]:
# Import dependencies
import pandas as pd
import datetime as dt

In [193]:
# Grab the CSV
athelete_stats_df = pd.read_csv("initial_unclean_csv/unclean_data.csv", encoding="utf8")

In [194]:
# Display as dataframe
athelete_stats_df.head()

,player_name,team,league,position,jersey_number,goals,assists,yellow_cards,red_cards,shots,shots_on_goal,saves,clean_sheets,goals_against,photo_url,height,weight,birthdate,nationality
0,Sophia Smith,Portland Thorns FC,nwsl,Forward,9.0,11,5,2,0,75,39,0,2,16,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,"5' 6""",121 lbs,2000-08-10T07:00Z,USA
1,Sophie Schmidt,Houston Dash,nwsl,Midfielder,13.0,0,0,3,0,17,3,0,8,15,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,"5' 8""",157 lbs,1988-06-28T07:00Z,Canada
2,Christine Sinclair,Portland Thorns FC,nwsl,Forward,12.0,3,1,0,0,27,10,0,1,12,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,"5' 9""",148 lbs,1983-06-12T07:00Z,Canada
3,Abby Erceg,Racing Louisville FC,nwsl,Defender,20.0,2,0,1,0,15,6,0,6,22,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,"5' 10""",150 lbs,1989-11-20T08:00Z,New Zealand
4,Nadia Nadim,Racing Louisville FC,nwsl,Forward,10.0,1,0,0,0,14,5,0,0,5,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,"5' 9""",NaN,1988-01-02T08:00Z,Denmark


### Cleaning:
---

#### Convert Height

In [206]:
# Split current heights
split_heights = athelete_stats_df["height"]
split_heights = split_heights.str.split("'", n=1, expand=True)

In [207]:
# Remove everthing but the numbers from the feet
# Convert to float for math
heights_feet = split_heights[0].astype("float")
heights_feet.head(5)

0    5.0
1    5.0
2    5.0
3    5.0
4    5.0
Name: 0, dtype: float64

In [208]:
# Remove everything but the nubmers from the inches
heights_inches = split_heights[1]

split_heights_inches = heights_inches.str.split('"', n=1, expand=True)

# Convert to float for math
split_heights_inches = split_heights_inches[0].astype("float")
heights_inches = split_heights_inches
heights_inches.head(5)

0     6.0
1     8.0
2     9.0
3    10.0
4     9.0
Name: 0, dtype: float64

In [209]:
# Converting inches to feet
inches_in_feet = []
for measurement in heights_inches:
    cur_feet = measurement/12
    inches_in_feet.append(cur_feet)
   

In [210]:
# Create dataframe with new heights colums, both in feet
split_heights_in_ft = {
    "heights_ft1": heights_feet,
    "heights_ft2": inches_in_feet
}
split_heights_in_ft_df = pd.DataFrame(split_heights_in_ft)
split_heights_in_ft_df.head(5)

,heights_ft1,heights_ft2
0,5.0,0.500000
1,5.0,0.666667
2,5.0,0.750000
3,5.0,0.833333
4,5.0,0.750000


In [211]:
# Adding feet and converted inches column to get new total feet measurments column with decimals
new_height = split_heights_in_ft_df["heights_ft1"] + split_heights_in_ft_df["heights_ft2"]

# Round the combined height
new_height = round(new_height,2)

In [212]:
# Display cleaned data
new_height.head(5)

0    5.50
1    5.67
2    5.75
3    5.83
4    5.75
dtype: float64

#### Convert Weight

In [213]:
# Grab uncleaned weight
uncleaned_weight = athelete_stats_df["weight"]
uncleaned_weight.head(5)

0    121 lbs
1    157 lbs
2    148 lbs
3    150 lbs
4        NaN
Name: weight, dtype: object

In [214]:
# Split weights
split_weights = uncleaned_weight.str.split(' ', n=1, expand=True)
split_weights.head(5)

,0,1
0,121,lbs
1,157,lbs
2,148,lbs
3,150,lbs
4,NaN,NaN


In [215]:
# Display cleaned data
new_weight = split_weights[0]
new_weight.head(5)

0    121
1    157
2    148
3    150
4    NaN
Name: 0, dtype: object

#### Convert Dates

In [216]:
# Split Current Dates
split_birthdates = athelete_stats_df["birthdate"]
split_birthdates = split_birthdates.str.split("T", n=1, expand=True)

In [217]:
# Select only the part we want to keep
split_birthdates = split_birthdates [0]
split_birthdates.head(5)

0    2000-08-10
1    1988-06-28
2    1983-06-12
3    1989-11-20
4    1988-01-02
Name: 0, dtype: object

In [218]:
# Convert the dates to datetime
split_birthdates = pd.to_datetime(split_birthdates)
split_birthdates.head(5)

0   2000-08-10
1   1988-06-28
2   1983-06-12
3   1989-11-20
4   1988-01-02
Name: 0, dtype: datetime64[ns]

In [219]:
# Check dates converted to timestamps
type (split_birthdates[0])

pandas._libs.tslibs.timestamps.Timestamp

In [220]:
# Display cleaned data
new_birthdates = split_birthdates
new_birthdates.head(5)

0   2000-08-10
1   1988-06-28
2   1983-06-12
3   1989-11-20
4   1988-01-02
Name: 0, dtype: datetime64[ns]

#### Remove "Goals Against" and "Clean Sheets" for Non-Goalies

In [221]:
# Remove goals against and clean sheets for non-goalies
new_goals_against = []
new_clean_sheets = []

for index, row in athelete_stats_df.iterrows():
    cur_position = row["position"]
    cur_goals_against = row["goals_against"]
    cur_clean_sheets = row["clean_sheets"]
    
    if cur_position == "Goalkeeper":
        new_goals_against.append(cur_goals_against)
        new_clean_sheets.append(cur_clean_sheets)
    else:
        new_goals_against.append(0)
        new_clean_sheets.append(0)


#### Creating Cleaned Dataframe

In [222]:
# Remove unclean sections from dataframe
athelete_stats_df = athelete_stats_df.drop(["birthdate"], axis=1)
athelete_stats_df = athelete_stats_df.drop(["height"], axis=1)
athelete_stats_df = athelete_stats_df.drop(["weight"], axis=1)
athelete_stats_df = athelete_stats_df.drop(["goals_against"], axis=1)
athelete_stats_df = athelete_stats_df.drop(["clean_sheets"], axis=1)
athelete_stats_df.head()

,player_name,team,league,position,jersey_number,goals,assists,yellow_cards,red_cards,shots,shots_on_goal,saves,photo_url,nationality
0,Sophia Smith,Portland Thorns FC,nwsl,Forward,9.0,11,5,2,0,75,39,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,USA
1,Sophie Schmidt,Houston Dash,nwsl,Midfielder,13.0,0,0,3,0,17,3,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,Canada
2,Christine Sinclair,Portland Thorns FC,nwsl,Forward,12.0,3,1,0,0,27,10,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,Canada
3,Abby Erceg,Racing Louisville FC,nwsl,Defender,20.0,2,0,1,0,15,6,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,New Zealand
4,Nadia Nadim,Racing Louisville FC,nwsl,Forward,10.0,1,0,0,0,14,5,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,Denmark


In [223]:
# Add clean sections to dataframe
athelete_stats_df["birthdate"] = new_birthdates
athelete_stats_df["height_(ft)"] = new_height
athelete_stats_df["weight_(lbs)"] = new_weight
athelete_stats_df["goals_against"] = new_goals_against
athelete_stats_df["clean_sheets"] = new_clean_sheets
athelete_stats_df.head()

,player_name,team,league,position,jersey_number,goals,assists,yellow_cards,red_cards,shots,shots_on_goal,saves,photo_url,nationality,birthdate,height_(ft),weight_(lbs),goals_against,clean_sheets
0,Sophia Smith,Portland Thorns FC,nwsl,Forward,9.0,11,5,2,0,75,39,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,USA,2000-08-10,5.50,121,0,0
1,Sophie Schmidt,Houston Dash,nwsl,Midfielder,13.0,0,0,3,0,17,3,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,Canada,1988-06-28,5.67,157,0,0
2,Christine Sinclair,Portland Thorns FC,nwsl,Forward,12.0,3,1,0,0,27,10,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,Canada,1983-06-12,5.75,148,0,0
3,Abby Erceg,Racing Louisville FC,nwsl,Defender,20.0,2,0,1,0,15,6,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,New Zealand,1989-11-20,5.83,150,0,0
4,Nadia Nadim,Racing Louisville FC,nwsl,Forward,10.0,1,0,0,0,14,5,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,Denmark,1988-01-02,5.75,NaN,0,0


In [226]:
# Reorder dataframe
athelete_stats_df = athelete_stats_df[["player_name",
                                       "team",
                                       "league",
                                       "position",
                                       "jersey_number",
                                       "goals",
                                       "assists",
                                       "yellow_cards",
                                       "red_cards",
                                       "shots",
                                       "shots_on_goal",
                                       "saves",
                                       "clean_sheets",
                                       "goals_against",
                                       "photo_url",
                                       "height_(ft)",
                                       "weight_(lbs)",
                                       "birthdate",
                                       "nationality"
                                       ]]
athelete_stats_df.head()

,player_name,team,league,position,jersey_number,goals,assists,yellow_cards,red_cards,shots,shots_on_goal,saves,clean_sheets,goals_against,photo_url,height_(ft),weight_(lbs),birthdate,nationality
0,Sophia Smith,Portland Thorns FC,nwsl,Forward,9.0,11,5,2,0,75,39,0,0,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,5.50,121,2000-08-10,USA
1,Sophie Schmidt,Houston Dash,nwsl,Midfielder,13.0,0,0,3,0,17,3,0,0,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,5.67,157,1988-06-28,Canada
2,Christine Sinclair,Portland Thorns FC,nwsl,Forward,12.0,3,1,0,0,27,10,0,0,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,5.75,148,1983-06-12,Canada
3,Abby Erceg,Racing Louisville FC,nwsl,Defender,20.0,2,0,1,0,15,6,0,0,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,5.83,150,1989-11-20,New Zealand
4,Nadia Nadim,Racing Louisville FC,nwsl,Forward,10.0,1,0,0,0,14,5,0,0,0,https://d2nkt8hgeld8zj.cloudfront.net\images\2...,5.75,NaN,1988-01-02,Denmark


### Creating Dataframes for SQL Tables:
---

#### Creating Leagues Dataframe

In [245]:
leagues = athelete_stats_df["league"].unique()
leagues_df = pd.DataFrame(leagues)
leagues_df = leagues_df.reset_index()
leagues_df = leagues_df.rename(columns={"index":"league_id", 0:"league"})
leagues_df["league_id"] = leagues_df["league_id"]+1
leagues_df["league_id"] = ["league" + str(league_id) for league_id in leagues_df["league_id"]]
leagues_df

,league_id,league
0,league1,nwsl


### Exports:
---

In [ ]:
# Export CSV 1-Player Data

In [ ]:
# Export CSV 2-Teams Data

In [ ]:
# Export CSV 3-Leagues Data

In [ ]:
# Export CSV 4-Positions Data

In [ ]:
# Export CSV 5-Nationalities Data